In [1]:
%load_ext graph_notebook.magics

The graph_notebook.magics extension is already loaded. To reload it, use:
  %reload_ext graph_notebook.magics


In [2]:
%graph_notebook_host growsmart-neptune.cluster-custom-cgogeml0cuty.eu-west-3.neptune.amazonaws.com

set host to growsmart-neptune.cluster-custom-cgogeml0cuty.eu-west-3.neptune.amazonaws.com


In [3]:
#!pip install dgl
#!pip install -U 'jsonschema<4.0'
#!pip install torch>=1.9.0

In [4]:
import pandas as pd
import torch
import dgl
import numpy as np

## Generating the embeddings

https://julsimon.medium.com/a-primer-on-graph-neural-networks-with-amazon-neptune-and-the-deep-graph-library-5ce64984a276

In [6]:
%%oc
MATCH (n)
RETURN n

In [7]:
%%oc --store-to edges
MATCH (n)-[e]->(m)
RETURN n['~id'] AS start, m['~id'] AS end

In [8]:
edges_tuples = [(ed['start'], ed['end']) for ed in edges['results']]
edges_tuples[:5]

[('SENSOR_p0:33:18:00:20:11', 'EVENT__16511f1d-cfca-5383-aba0-6a388d52a8fe'),
 ('PLANT_6f5dbe0d-b7c4-4608-a511-a8cc521c40b8', 'SPECIES_39210'),
 ('PLANT_87a5f59e-a955-4c46-8393-7e678de72f46', 'SPECIES_41305'),
 ('GARDEN_16FFB3', 'PLANT_29b24e45-5e2f-48fc-81bf-0ce7428f7d1d'),
 ('GARDEN_4D2E71', 'PLANT_36c27bcf-fb3f-44f7-827b-6731ebabc41d')]